# Telstra Kaggle Competition

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
cd C:\Users\JChung\Desktop\kaggle comp\telstra

C:\Users\JChung\Desktop\kaggle comp\telstra


In [8]:
train = pd.read_csv("train.csv",index_col=0)
test = pd.read_csv("test.csv",index_col=0)

In [9]:
test.sort_index().head()

,location
id,
2,location 474
3,location 64
4,location 645
7,location 638
9,location 1100


In [10]:
train.sort_index().head()

,location,fault_severity
id,,
1,location 601,1
5,location 460,0
6,location 332,1
8,location 243,0
13,location 418,0


In [14]:
eventType = pd.read_csv("event_type.csv", index_col=0)
eventTypeVectorized = pd.get_dummies(eventType).groupby(eventType.index).sum()
severityType = pd.read_csv('severity_type.csv', index_col=0)
severityTypeVectorized = pd.get_dummies(severityType).groupby(severityType.index).sum()
logFeature = pd.read_csv('log_feature.csv', index_col=0)
logFeatureVectorized = pd.get_dummies(logFeature).groupby(logFeature.index).sum()
resourceType = pd.read_csv('resource_type.csv', index_col=0)
resourceTypeVectorized = pd.get_dummies(resourceType).groupby(resourceType.index).sum()

In [15]:
trainmerged = train.join(eventTypeVectorized).join(severityTypeVectorized).join(logFeatureVectorized).join(resourceTypeVectorized)

In [16]:
testmerged = test.join(eventTypeVectorized).join(severityTypeVectorized).join(logFeatureVectorized).join(resourceTypeVectorized)

In [33]:
len(testmerged.location.unique())

1039

In [34]:
len(trainmerged.location.unique())

929

In [18]:
trainmerged.head()

,location,fault_severity,event_type_event_type 1,event_type_event_type 10,event_type_event_type 11,event_type_event_type 12,event_type_event_type 13,event_type_event_type 14,event_type_event_type 15,event_type_event_type 17,...,resource_type_resource_type 1,resource_type_resource_type 10,resource_type_resource_type 2,resource_type_resource_type 3,resource_type_resource_type 4,resource_type_resource_type 5,resource_type_resource_type 6,resource_type_resource_type 7,resource_type_resource_type 8,resource_type_resource_type 9
id,,,,,,,,,,,,,,,,,,,,,
14121,location 118,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9320,location 91,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
14394,location 152,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
8218,location 931,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
14804,location 120,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [19]:
#Lets try Logistic Regression
from sklearn.linear_model import LogisticRegression

In [21]:
#Prepare training data
X_train = X_train.drop(['fault_severity','location',axis=1)
Y_train = trainmerged.fault_severity

In [23]:
#Prepare test data
X_test = pd.get_dummies(testmerged)
ids = testmerged.index

In [40]:
print X_train.shape
print Y_train.shape

(7381, 1581)
(7381L,)


In [41]:
print X_test.shape

(11171, 1581)


In [37]:
for n in X_test.columns:
    if n not in X_train.columns:
        X_train[n] = np.zeros(X_train.shape[0])

In [39]:
for n in X_train.columns:
    if n not in X_test.columns:
        X_test[n] = np.zeros(X_test.shape[0])

In [26]:
ids

Int64Index([11066, 18000, 16964,  4795,  3392,  3795,  2881,  1903,  5245,
             6726, 
            ...
             6446,  9063, 11825,  8982,  2356, 14806,  1825,  2374,  7277,
             9886],
           dtype='int64', name=u'id', length=11171)

In [42]:
log = LogisticRegression()
log.fit(X_train,Y_train)
Y_pred = log.predict(X_test)

In [55]:
asdf = {"Predict":Y_pred}

In [95]:
data = []
for n in Y_pred:
    x = str(n)
    data.append(x)

In [96]:
data = pd.DataFrame({"predict":data})

In [97]:
data2 = pd.get_dummies(data)

In [98]:
data2['id'] = ids

In [101]:
data2 = data2[['id','predict_0','predict_1','predict_2']].set_index('id')

In [107]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11171 entries, 11066 to 9886
Data columns (total 3 columns):
predict_0    11171 non-null float64
predict_1    11171 non-null float64
predict_2    11171 non-null float64
dtypes: float64(3)
memory usage: 349.1 KB


In [103]:
data2.to_csv('submission.csv')

'head' is not recognized as an internal or external command,
operable program or batch file.


In [53]:
#Create Dummy Variables
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categorical_features=[0])
ohe = ohe.fit_transform(Y_pred).toarray()

In [54]:
ohe

array([[ 1.,  2.,  0., ...,  1.,  0.,  0.]])

In [ ]:
data = pd.DataFrame({ids})

In [45]:
dict?